# Context Management:

Probably the most common use of context managers is to manage resources (i.e. allocating and releasing resoures), such as connections to files or databases. In many cases, resources need to be handled properly (i.e. closed properly and promptly) or the underlying operating systems may balk, produce errors, or just cry piteously.

Context managers are also commonly used in setup and teardown code OR any set of steps that should happen before or after a process.

Let's dive right in an use a context manager and talk about what happens under the hood.

*Hat tip to Luciano Ramalho, author of Fluent Python*

In [ ]:
with open('war_of_the_worlds.txt') as fin:
    first_line = fin.readline()
    # additional code goes here...
    
len(first_line)    


### What just happened?

* `with` calls the `*.__enter__()` method
* in this case, `open`'s `*.__enter__()` method returns the file object
* the label `fin` is bound to the file object
* when control flow exits the `with` block, the `*.__exit__()` method is called

We can print the variable `first_line`

In [ ]:
print(first_line)

We can access the label `fin` and see that `fin` points to an input-output file object.

In [ ]:
repr(fin)

If we look closely, we will see that sure enough, the file object is now closed. 

In [ ]:
fin.closed

Despite being closed (i.e. not able to access content in the file), we can still access certain attributes about the file object.

In [ ]:
fin.encoding

Just to confirm that we can't read from the file any longer...

In [ ]:
fin.readline()

How does this compare with opening a file the naive way?

In [ ]:
fin = open("war_of_the_worlds.txt")
line_one = fin.readline()
# additional code goes here...
fin.close()

len(line_one)

By comparison, this seems like it only saves us one line AND yet basically does the same thing...

OR does it?

Let's cause an `Error` and see what happens...

In [ ]:
error_fin_1 = open("war_of_the_worlds.txt")
line_one = error_fin_1.readline()
# additional code goes here...
# blah, blah
# blah, blah
raise Exception  # an ERROR occurs here...
error_fin_1.close()

len(line_one)

If we check... we see that the `Error` prevented our file from **closing**.

In [ ]:
error_fin_1.closed

Raising a similar `Error` within a context manager on the other hand...

In [ ]:
with open('war_of_the_worlds.txt') as error_fin_2:
    first_line = error_fin_2.readline()
    # additional code goes here...
    # code block, blah, blah
    # code block, blah, blah
    raise Exception # an ERROR occurs here...
    
len(first_line) 

...does **NOT** prevent the file from being closed.

In [ ]:
error_fin_2.closed

### the 'as' clause is optional and will not be used in all context managers
* Some context managers, such as `file objects`... return self 
* Some context managers return `None`


# Looking Glass

Let's take a look at the `Looking Glass class` that Luciano mentions in his book.

Notice, this `class` has an:

* `__enter__()` method
* `__exit__()` method


```python
class LookingGlass:

    def __enter__(self):
        '''Save a reference to sys.stdout.write and overwrite
        it with a new version.'''
        
        import sys
        self.original_write = sys.stdout.write
        sys.stdout.write = self.reverse_write
        return 'JABBERWOCKY'

    def reverse_write(self, text):
        '''Write text, in reverse order to standard out'''
        
        self.original_write(text[::-1])

    def __exit__(self, exc_type, exc_value, traceback):
        '''Set everything back to normal'''
        
        import sys
        sys.stdout.write = self.original_write
        if exc_type is ZeroDivisionError:
            print('Please do not divide by zero!')
            return True
```

We have a copy of this `class` in the accompanying file: `mirror.py`

In [ ]:
from mirror import LookingGlass

In [ ]:
with LookingGlass() as cm_obj:
    print('This is backwards!')
    raise Exception
    

In [ ]:
print('hello')
print('Elvis has left the building')
    

In [ ]:
print('hello')

In [ ]:
with LookingGlass() as cm_obj:
    print(cm_obj)
    print('reversed')
    

In [ ]:
print(cm_obj)

In [ ]:
print('Back from wonderland')

## Another example... database connection

In [ ]:
import sqlite3

class DatabaseConnector:
    def __init__(self, database):
        self.database = database
        
    def __enter__(self):
        self.conn = sqlite3.connect(self.database)
        return self.conn
    
    def __exit__(self, exc_type, exc_value, exc_traceback):
        self.conn.close()
        
database = 'sample.db'
with DatabaseConnector(database) as conn:
    cursor = conn.cursor()
    

## Using a context manager without a with block

In [ ]:
from mirror import LookingGlass
manager = LookingGlass()
manager

In [ ]:
monster = manager.__enter__()
print(monster == 'JABBERWOCKY')


In [ ]:
print('backwards')

In [ ]:
manager.__exit__(None, None, None)

In [ ]:
print('hello')

# Where are context managers used?

* `Path` objects in `pathlib`
* `ZipFile` objects in `zipfile`
* `Popen` objects in `subprocess`
* `TarFile` objects in `tarfile`
* `Lock` objects in `threading`
* `Telnet` objects in `telnetlib`
* `StringIO` objects in `io`
* `patch` objects in `mock`

and more!

# contextlib

In [ ]:
from contextlib import contextmanager

@contextmanager
def open_file(path, mode):
    print('about to open file')
    the_file = open(path, mode)
    yield the_file
    print('closing file')
    the_file.close()

In [ ]:
files = []

for x in range(10):
    with open_file('foo.txt', 'w') as infile:
        files.append(infile)

In [ ]:
for f in files:
    if not f.closed:
        print('not closed')

In [ ]:
files